In [122]:
%matplotlib inline
import json
import re
import numpy as np
import matplotlib.pyplot as plt
with open('../classifications/classifications.json') as f:
    classifications = json.load(f)
with open('../classifications/allSubjectIds.json') as f:
    subjects = np.fromiter(map(int, json.load(f)), dtype=int)
with open('../classifications/subjectForClassification.csv') as f:
    classificationToSubjectMap = np.fromiter(map(int, f.read().split('\n')), dtype=int)
    
with open('../classifications/galaxy-builder-subjects_30-5-18.csv') as f:
    subs = f.read().split('\n')
subIds = np.array([int(s.split(',')[0]) for s in subs[1:-1]])
subUrls = [re.search(r'"(https?://panoptes-uploads\.[\w/._-]+\.png)"', s).group(1) for s in subs[1:-1]]

In [121]:
print(subs[300])
print(re.search(r'"(https?://panoptes-uploads\.[\w/._-]+\.png)"', subs[1]).group(1))

21686570,5590,5366,21182,"{""ra"":""175.265320"",""Run"":""3525"",""dec"":""10.225102"",""Field"":""136"",""Rerun"":""301"",""Camcol"":""4"",""NSA id"":""693002"",""#models"":[{""frame"":0,""model"":""GALAXY_BUILDER_DIFFERENCE""},{""frame"":2,""model"":""GALAXY_BUILDER_MODEL""}],""redshift"":""0.020862"",""Common name"":""Unknown"",""SDSS dr7 id"":""587734893287964700"",""Url to view"":""[skyserver link](+tab+http://skyserver.sdss.org/dr14/en/tools/chart/navi.aspx?ra=175.26531982421875\u0026dec=10.225102424621582\u0026opt=F)"",""#isModelling"":true,""Sersic axis ratio"":""0.732124"",""Estimated distance"":""0.02057170309126377 * c / H_0"",""Petrosian radius (degrees)"":""29.038961""}","{""0"":""https://panoptes-uploads.zooniverse.org/production/subject_location/34c471c2-099c-4628-aa77-dff5695a9934.json"",""1"":""https://panoptes-uploads.zooniverse.org/production/subject_location/bab7e603-cc20-448f-8012-0a82bf0bf97f.png"",""2"":""https://panoptes-uploads.zooniverse.org/production/subjec

In [150]:
import io
from PIL import Image
from urllib.request import urlopen
plt.figure(figsize=(10, 10))
for chosenId in subjects[100:]:
    i = np.where(subIds == chosenId)[0][0]
    chosenUrl = subUrls[i]
    file = io.BytesIO(urlopen(chosenUrl).read())
    img = Image.open(file)
    plt.imshow(img, cmap='bone')
    plt.xticks([])
    plt.yticks([])
    cls = [classifications[i] for i in np.where(classificationToSubjectMap == chosenId)[0]]
    for c in cls:
        if len(c[3]['value'][0]['value']) > 0:
            for spiral in c[3]['value'][0]['value']:
                points = np.asarray([[i['x'], i['y']] for i in spiral['points']])
                plt.plot(points[:, 0], points[:, 1],alpha=0.2, linewidth=5)

    plt.savefig('outputs/subject-{}.png'.format(chosenId))
    plt.clf()

https://panoptes-uploads.zooniverse.org/production/subject_location/b52c16b0-dae3-464b-bfb6-b50bf6c97560.png
https://panoptes-uploads.zooniverse.org/production/subject_location/85d00193-63af-453e-925b-ecde59f04b80.png
https://panoptes-uploads.zooniverse.org/production/subject_location/51e02b1d-15b4-4756-8cf8-cfc6bba596b0.png
https://panoptes-uploads.zooniverse.org/production/subject_location/54f0a93a-0148-4a16-afae-f9fc678cd9ab.png
https://panoptes-uploads.zooniverse.org/production/subject_location/dec03751-291c-4374-8d75-93e4d0c844a1.png
https://panoptes-uploads.zooniverse.org/production/subject_location/7cc5347c-b0ca-4ca8-babe-3e838cf03b9a.png
https://panoptes-uploads.zooniverse.org/production/subject_location/8cd54272-a2bb-4591-ad4d-afe91fa6d430.png
https://panoptes-uploads.zooniverse.org/production/subject_location/2ff18efe-bb44-4f55-9389-7bb4340ec878.png
https://panoptes-uploads.zooniverse.org/production/subject_location/9dd086eb-4fc0-4797-8452-5f447987f64c.png
https://panoptes-up